In [1]:
!pip install importlib-metadata==4.13.0
!pip install dionysus
!pip install gudhi
!pip install networkx
!pip install persim
!pip install xarray
!pip install validclust
!pip install zarr
!pip install POT==0.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.1.0
    Uninstalling importlib-metadata-5.1.0:
      Successfully uninstalled importlib-metadata-5.1.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 4.6 MB/s 
  Created wheel for dionysus: filename=dionysus-2.0.8-cp38-cp38-linux_x86_64.whl size=319865 sha256=14f75500379917a68c14bb82ce7db901e005ab296208f34d00059d8a4567cf79
  Stored in directory: /root/.cache/pip/wheels/0e/6a/ce/a54c9248873852edfaf8e6e801db84c2d2c64f797fc07a68da
Successfully built dionysus
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 31.6 MB 89.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/pub

In [2]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive',force_remount=True)
import os
os.chdir("/content/drive/My Drive/NASA")
!ls -ltr
from __future__ import print_function
import pickle
#import zigzagtools as zzt
import numpy as np
from numpy import linalg as LA
import pandas as pd
#import networkx as nx
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist
import dionysus as d
import matplotlib.pyplot as plt
import time
import netCDF4
import numpy as np
import gudhi as gd
#import networkx as nx
from persim import PersImage
import sys
import time
from validclust import dunn, cop, ValidClust
import itertools
import os
# added
import ot
from gudhi.wasserstein import wasserstein_distance as wd
#import regrid_hp as rhp
#import cartopy.crs as ccrs
#import healpy as hp
import xarray as xr
import mpl_toolkits
#from cartopy.io import shapereader
#from mpl_toolkits.basemap import Basemap
#import ocw.utils as utils
#import ocw.data_source.local as local
#import ocw.dataset_processor as dsp
#from ocw.dataset import Bounds as Bounds
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score, pairwise_distances
from sklearn.cluster import KMeans, AgglomerativeClustering
from persim import wasserstein
from sklearn.cluster import DBSCAN
import zarr
import seaborn as sns
sns.set_theme()

Mounted at /content/drive
total 7861
-rw------- 1 root root    1667 Oct  5 18:53  zigzagtools.py
-rw------- 1 root root   85452 Oct  5 18:53  zigzag_PD_eps_1.pdf
-rw------- 1 root root       0 Oct  5 18:53  VRrips.pk
-rw------- 1 root root  133051 Oct  5 18:53  PIs_labelled_modulo_month.pdf
-rw------- 1 root root  132779 Oct  5 18:53  PIs_labelled_modulo_month_1.pdf
-rw------- 1 root root  105850 Oct  5 18:53  PIs_labelled_modulo_month_0.pdf
-rw------- 1 root root  127787 Oct  5 18:53  PIs_labelled_1.pdf
-rw------- 1 root root  116243 Oct  5 18:53  PIs_labelled_0.pdf
-rw------- 1 root root  253295 Oct  5 18:53  PClouds.pdf
-rw------- 1 root root 1682280 Oct  5 18:53  Merra_t2m_freudenthal_Oct_2022.ipynb
-rw------- 1 root root  324306 Oct  5 18:53  Merra_t2m_freudenthal.ipynb
-rw------- 1 root root  323836 Oct  5 18:53  MERRA2_t2m_GP_20210201-20210228.nc
-rw------- 1 root root 1690684 Oct  5 18:53  MERRA2_t2m_CONUS_20210201-20210228.nc
-rw------- 1 root root  527019 Oct  5 18:53  MERRA2

In [3]:
path = os.getcwd()
#%% Parameters 
#nameFolderCloud = path + '/PointCloud/5_Dimensions/datafullpc'  # Example 1 (Dorcas)  (sizeWindow = 10) * It has not any interesting structures
#nameFolderCloud = path + '/PointCloud/FIG8Saved_40/File'  # Example 2  (sizeWindow = 12)
#nameFolderCloud = path + '/../Example_POINT_CLOUD/PointCloud/FIG8Saved_60/File'  # Example 3  (sizeWindow = 12)
#scaleParameter = 1.0 # [Example 2 and 3] Maximum Scale Parameter for Filtration (Depends on distance between points)
#scaleParameter = 9.0 # [Example 1] Maximum Scale Parameter for Filtration (Depends on distance between points)
maxDimHoles = 2 # Maximum Dimension of Holes (It means.. 0 and 1)
sizeWindow = 28 # [Example 2 and 3] Number of Point-Clouds (i.e. timestamps) 
#sizeWindow = 10 # [Example 1] Number of Point-Clouds (i.e. timestamps) 

In [4]:
def extract_PD_arrays_with_cubical_complex(data, infinity = 300.):
    cubical_complex = gd.CubicalComplex(top_dimensional_cells=data)
    PD_CC = cubical_complex.persistence()
    dim = np.array([i[0] for i in PD_CC])
    birth = np.array([i[1][0] for i in PD_CC])
    death = np.array([i[1][1] for i in PD_CC])
    death[death==np.Infinity] = infinity
    n0 = dim[dim==0].size
    n1 = dim[dim==1].size
    pd0 = np.zeros([n0, 2])
    pd1 = np.zeros([n1, 2])
    pd0[:,0] = birth[dim==0]
    pd0[:,1] = death[dim==0]
    pd1[:,0] = birth[dim==1]
    pd1[:,1] = death[dim==1]
    return cubical_complex, pd0, pd1

def calculate_wasserstein_distance(pd0_array1, pd0_array2, pd1_array1, pd1_array2, order=2):
    return wd(pd0_array1, pd0_array2, order=order) + wd(pd1_array1, pd1_array2, order=order)

def extract_PD_arrays_with_freudenthal_triangulation(data, lower =240., infinity = 300.):
    
    f_lower_star = d.fill_freudenthal(np.array(data, dtype='float64'))
    p = d.homology_persistence(f_lower_star)
    dgms = d.init_diagrams(p, f_lower_star)
    birth0=np.array([dgm.birth for dgm in dgms[0]])
    death0=np.array([dgm.death for dgm in dgms[0]])
    death0[death0==np.Infinity] = infinity 
    birth1=np.array([dgm.birth for dgm in dgms[1]])
    death1=np.array([dgm.death for dgm in dgms[1]])
    death1[death1==np.Infinity] = infinity
   
    return [[birth0,death0],[birth1,death1]]

def plot_PDs (pd0_array, pd1_array, titles, lower_bound=240, upper_bound=300):
    fig = plt.figure(figsize=(25,15))
    nday = len(pd0_array)
    for iday in np.arange(nday):
        ax = fig.add_subplot(4,7,1+iday)
        ax.scatter(pd0_array[iday][:,0], pd0_array[iday][:,1], color='r', s=5, label = '0')
        ax.scatter(pd1_array[iday][:,0], pd1_array[iday][:,1], marker='^', s=5,
               facecolors='none', edgecolor='blue', label = '1')
        
        ax.plot(np.arange(lower_bound, upper_bound), np.arange(lower_bound, upper_bound), '-', color='grey', lw=0.5)
        ax.set_xlim([lower_bound, upper_bound])
        ax.set_ylim([lower_bound, upper_bound])
        ax.set_title(titles[iday])
        #ax.set_xlabel('Birth')
        #ax.set_ylabel('Death')
        #ax.set_xticks(np.arange(4)*20+240)
        #ax.set_xticklabels(np.repeat('',4))
        #ax.set_yticks(np.arange(4)*20+240)
        #ax.set_yticklabels(np.repeat('',4))

def plot_Dionysus_PDs(pd0_array, pd1_array, titles, lower_bound=240, upper_bound=300):
    fig = plt.figure(figsize=(30,20))
    nday = len(pd0_array)
    for iday in np.arange(nday):
        ax = fig.add_subplot(4,7,1+iday)
        ax.scatter(pd0_array[iday][0], pd0_array[iday][1], color='r', s=5, label = '0')
        ax.scatter(pd1_array[iday][0], pd1_array[iday][1], marker='^', s=5,
               facecolors='none', edgecolor='blue', label = '1')
        
        ax.plot(np.arange(lower_bound, upper_bound), np.arange(lower_bound, upper_bound), '-', color='grey', lw=0.5)
        ax.set_xlim([lower_bound, upper_bound])
        ax.set_ylim([lower_bound, upper_bound])
        ax.set_title(titles[iday])
        ax.set_xlabel('Birth')
        ax.set_ylabel('Death')
    plt.savefig("pds.pdf",bbox_inches='tight')
    plt.show()   

def plot_Dionysus_PDs_by_Cluster(pd0_array, pd1_array, titles, cluster_labels, lower_bound=240, upper_bound=300):
    fig = plt.figure(figsize=(30,20))
    nday = len(pd0_array)
    idx0= np.where(cluster_labels==0)[0]
    idx1= np.where(cluster_labels==1)[0]
    
    ax = fig.add_subplot(2,2,1)
    for iday in idx0:    
        ax.scatter(pd0_array[iday][0], pd0_array[iday][1], color='r', s=5, label = '0')
    ax = fig.add_subplot(2,2,2)
    for iday in idx0:    
        ax.scatter(pd1_array[iday][0], pd1_array[iday][1], marker='^', s=5,
            facecolors='none', edgecolor='blue', label = '1')
        
    ax.plot(np.arange(lower_bound, upper_bound), np.arange(lower_bound, upper_bound), '-', color='grey', lw=0.5)
    ax.set_xlim([lower_bound, upper_bound])
    ax.set_ylim([lower_bound, upper_bound])
    #ax.set_title(titles[iday])
    ax.set_xlabel('Birth')
    ax.set_ylabel('Death')
    
    ax = fig.add_subplot(2,2,3)
    for iday in idx1:    
        ax.scatter(pd0_array[iday][0], pd0_array[iday][1], color='r', s=5, label = '0')
    ax = fig.add_subplot(2,2,4)
    for iday in idx1:    
        ax.scatter(pd1_array[iday][0], pd1_array[iday][1], marker='^', s=5,
               facecolors='none', edgecolor='blue', label = '1')
        
    ax.plot(np.arange(lower_bound, upper_bound), np.arange(lower_bound, upper_bound), '-', color='grey', lw=0.5)
    ax.set_xlim([lower_bound, upper_bound])
    ax.set_ylim([lower_bound, upper_bound])
    #ax.set_title(titles[iday])
    ax.set_xlabel('Birth')
    ax.set_ylabel('Death')

def keyfunc(s):
    return s[0:5], int(s[5:7]), s[7], int(s[8:10])

def selectChannel(dir, channelNum):
    
    #tileList = [filename for filename in os.listdir(dir) if filename.startswith("Tile")]
    tempTileList = [filename for filename in os.listdir(dir) if filename.startswith("Tile")]

    print (len(tempTileList))
    tileList = []
    for i in tempTileList:
        temp = i.replace('.', '_')
        tempList = temp.split('_')
        tempList[1] = str(tempList[1].zfill(2))
        tempList[2] = str(tempList[2].zfill(2))
        result = '_'.join(tempList)
        result = result[0:10] + '.' + result[-4:]
        tileList.append(result)
    
    dict1 = dict(zip(tileList, tempTileList))
    print (dict1)
    tileList = sorted(tileList, key=keyfunc)
    print (len(tileList))
    # take first 80 tiles for test
    tileList = tileList[0:300]
    data = np.empty((len(tileList), 32, 32))

    for i in range(len(tileList)):
        tempData = zarr.open(dir + '//' + dict1[tileList[i]])[:, :, channelNum]
        print (dict1[tileList[i]])
        data[i, :, :] = tempData

    return data

def optimalDBSCAN(data, score_, eps, metric='euclidean', bMax=True, distType=0):
    eps_range=np.arange(eps['min'],eps['max'],eps['inc'])
    min_samples_range=range(2,10)
    if bMax==True:
        best_score =-1e9
    else:
        best_score =1e9
        
    eps_ask=0
    min_samples_ask=0
    clustering_ask=None
    print("score, best_score, eps, eps_ask, min_samples, min_samples_ask")
    for min_samples in min_samples_range:
     for eps in eps_range:
      clustering = DBSCAN(eps=eps, min_samples=min_samples, metric=metric)
      clustering.fit(data)
      if len(np.unique(clustering.labels_))>1:
        if distType==0:
            score=score_(data, clustering.labels_)
        elif distType==1:
            dist=pairwise_distances(data)
            score=score_(dist, clustering.labels_)
        elif distType==2:
            dist=pairwise_distances(data)
            score=score_(data, dist, clustering.labels_)
        if (score>best_score and bMax==True):
          best_score=score
          eps_ask=eps
          min_samples_ask=min_samples
          clustering_ask=clustering
        elif(score<best_score and bMax==False):
          best_score=score
          eps_ask=eps
          min_samples_ask=min_samples
          clustering_ask=clustering  
        print(score, best_score, eps, eps_ask, min_samples, min_samples_ask)
    return(best_score, eps_ask, min_samples_ask, clustering_ask)  

def optimalHClust(data, score_, metric='euclidean', bMax=True, distType=0):
    if bMax==True:
        best_score =-1e9
    else:
        best_score =1e9
    linkage_ask = ''  
    n_ask = 2
    clustering_ask = None
    print("score, best_score, i, n_ask, linkage, linkage_ask")
    for i in range(2, 7):
     for linkage in ['complete', 'average', 'single']: 
      clustering= AgglomerativeClustering(n_clusters=i, affinity=metric, linkage=linkage)
      #clustering = DBSCAN(eps=eps, min_samples=min_samples, metric=metric)
      clustering.fit(data)
      if len(np.unique(clustering.labels_))>1:
        if distType==0:
            score=score_(data, clustering.labels_)
        elif distType==1:
            dist=pairwise_distances(data)
            score=score_(dist, clustering.labels_)
        elif distType==2:
            dist=pairwise_distances(data)
            score=score_(data, dist, clustering.labels_)
        if (score>best_score and bMax==True):
          best_score=score
          linkage_ask=linkage
          n_ask=i
          clustering_ask=clustering
        elif(score<best_score and bMax==False):
          best_score=score
          linkage_ask=linkage
          n_ask=i 
          clustering_ask=clustering
        print(score, best_score, i, n_ask, linkage, linkage_ask)
    return(best_score, n_ask, linkage_ask, clustering_ask)

def persistence_image(dgm, resolution = [50,50], return_raw = False, normalization = True, bandwidth = 1., power = 1.):
    PXs, PYs = dgm[:, 0:1], dgm[:, 1:2]
    xm, xM, ym, yM = PXs.min(), PXs.max(), PYs.min(), PYs.max()
    x = np.linspace(xm, xM, resolution[0])
    y = np.linspace(ym, yM, resolution[1])
    X, Y = np.meshgrid(x, y)
    Zfinal = np.zeros(X.shape)
    X, Y = X[:, :, np.newaxis], Y[:, :, np.newaxis]

    P0, P1 = np.reshape(dgm[:, 0], [1, 1, -1]), np.reshape(dgm[:, 1], [1, 1, -1])

    # Compute image
    weight = np.abs(P1 - P0)
    distpts = np.sqrt((X - P0) ** 2 + (Y - P1) ** 2)

    if return_raw:
        lw = [weight[0, 0, pt] for pt in range(weight.shape[2])]
        lsum = [distpts[:, :, pt] for pt in range(distpts.shape[2])]
    else:
        weight = weight ** power
        Zfinal = (np.multiply(weight, np.exp(-distpts ** 2 / bandwidth))).sum(axis=2)

    output = [lw, lsum] if return_raw else Zfinal

    if normalization:
        norm_output = (output - np.min(output)) / (np.max(output) - np.min(output))
    else:
        norm_output = output

    return norm_output

In [8]:
fname = 'MERRA2_t2m_GP_20210201-20210228.nc'
fname2 = 'Sample Data'

ds = xr.open_dataset(fname)
#data = ds['t2m'].squeeze()

lon = ds.lon
lat = ds.lat
channelRange = range(16)

for channelNum in channelRange:
    print (channelNum)
    data = selectChannel(fname2, channelNum)

    nday = np.shape(data[0])[0]
    innerSizeWindow=7
    lower=int(np.floor(innerSizeWindow/2))
    upper=int(np.ceil(innerSizeWindow/2))

    #freudenthal_complex = []
    pd0 = []
    pd1 = []

    #freudenthal_complex_union = []
    pd0_union = []
    pd1_union = []
    for iday in np.arange(nday):
        # Build freudenthal_triangulation for individual day 
        p0, p1 = extract_PD_arrays_with_freudenthal_triangulation(data[iday,:])
        #freudenthal_complex.append(cc)
        pd0.append(p0)
        pd1.append(p1)
        # Build freudenthal_triangulation for the union of two consecutive days
        #if iday != nday-1:
        #p0, p1 = extract_PD_arrays_with_freudenthal_triangulation(0.5*data[iday,:]+0.5*data[iday+1,:])
        p0, p1 = extract_PD_arrays_with_freudenthal_triangulation(data[iday,:])
        
        #freudenthal_complex_union.append(cc)
        pd0_union.append(p0)
        pd1_union.append(p1)
    
    results = {}
    results['data'] = {}
    results['pis']= {}
    results['pds']= {}
    params = {}
    params['data'] = {}
    params['pis']= {}
    params['pds']= {}
    clusters = {}
    clusters['data'] = {}
    clusters['pis']= {}
    clusters['pds']= {}
    
    PIs = []
    for iday in np.arange(nday):
        xs= np.concatenate((pd0[iday][0][:], pd1[iday][0][:]), axis=0)
        ys= np.concatenate((pd0[iday][1][:], pd1[iday][1][:]), axis=0)
        D = np.transpose([xs,ys])
        PI = persistence_image(D, resolution =(100,100), power=0.01)
        PIs.append(PI.flatten())
    
    X = np.array(PIs)
    X -= np.min(X)
    X/= (np.max(X)-np.min(X))

    vclust = ValidClust(
    k=list(range(2, 8)),
    methods=['hierarchical', 'kmeans'],
    indices=['silhouette', 'calinski', 'davies', 'dunn', 'cop'])
    cvi_vals = vclust.fit_predict(X)

    keys = cvi_vals[2]['kmeans'].keys()
    for method in ['hierarchical','kmeans']:
        results['pis'][method]={}   
        for key in keys:
            if key=='davies':  
                idx=np.argmin([cvi_vals[x][method][key] for x in range(2,7)])
            else:
                idx=np.argmax([cvi_vals[x][method][key] for x in range(2,7)])
            #print(cvi_vals[2+idx][method][key])
            results['pis'][method][key]=cvi_vals[2+idx][method][key]
    
    wdist = np.zeros((nday,nday))
    for i in range(nday):
        for j in range(i+1):
            wdist[i,j] = wd(X[i,].reshape(1,10000), X[j,].reshape(1,10000), order=2) #wasserstein(Di, Dj) # where the format of PD is numpy matrix
            wdist[j,i] = wdist[i,j]
    
    wdist=(wdist-np.min(wdist))/(np.max(wdist)-np.min(wdist))
    eps={'min':0.01, 'max':10, 'inc': 0.05}
    results['pis']['DBSCAN']={}
    params['pis']['DBSCAN']={}
    clusters['pis']['DBSCAN'] = {}

    scores=['silhouette', 'calinski', 'davies', 'dunn', 'cop']
    bMax= True
    distType=0
    for score in scores:
        print('======' + score + '======' )
        if(score=='silhouette'):
            f= lambda data, labels: silhouette_score(data, labels)
        elif(score=='calinski'):
            f= lambda data, labels: calinski_harabasz_score(data, labels)
        elif(score=='davies'):
            f= lambda data, labels: davies_bouldin_score(data, labels)
            bMax=False
        elif(score == 'dunn'):
            f= lambda dist, labels: dunn(dist, labels)
            distType =1
        elif(score == 'cop'):
            f= lambda data, dist, labels: cop(data, dist, labels)
            distType =2
        results['pis']['DBSCAN'][score] = {}
        params['pis']['DBSCAN'][score] = {}
        clusters['pis']['DBSCAN'][score] = {}
        #[score_, eps_, min_samples_, clusters_]= optimalDBSCAN(data=X, score_=f, eps=eps, metric='euclidean', bMax=bMax, distType=distType)
        [score_, eps_, min_samples_, clusters_]= optimalDBSCAN(data=wdist, score_=f, eps=eps, metric='precomputed', bMax=bMax, distType=distType)
        
        results['pis']['DBSCAN'][score]= score_
        params['pis']['DBSCAN'][score]['eps'] = eps_
        params['pis']['DBSCAN'][score]['min_samples'] = min_samples_
        clusters['pis']['DBSCAN'][score] = clusters_
    
    results['pis']['Hierarchical']={}
    params['pis']['Hierarchical']={}
    clusters['pis']['Hierarchical'] = {}

    scores=['silhouette', 'calinski', 'davies', 'dunn','cop']
    bMax= True
    distType=0
    for score in scores:
        print('======' + score + '======' )
        if(score=='silhouette'):
            f= lambda data, labels: silhouette_score(data, labels)
        elif(score=='calinski'):
            f= lambda data, labels: calinski_harabasz_score(data, labels)
        elif(score=='davies'):
            f= lambda data, labels: davies_bouldin_score(data, labels)
            bMax=False
        elif(score == 'dunn'):
            f= lambda dist, labels: dunn(dist, labels)
            distType =1
        elif(score == 'cop'):
            f= lambda data, dist, labels: cop(data, dist, labels)
            distType =2
        results['pis']['Hierarchical'][score] = {}
        params['pis']['Hierarchical'][score] = {}
        clusters['pis']['Hierarchical'][score] = {}
        [score_, n, linkage, clusters_] = optimalHClust(data=wdist, score_=f, metric='precomputed', bMax=bMax, distType=distType)
        results['pis']['Hierarchical'][score]=score_
        params['pis']['Hierarchical'][score]['n']=n
        params['pis']['Hierarchical'][score]['linkage']=linkage
        clusters['pis']['Hierarchical'][score]=clusters_

    X = []
    dim=np.shape(data[0,:])
    for iday in np.arange(nday):
        X.append(np.array(data[iday,:]).reshape(dim[0]*dim[1]))
    
    X -= np.min(X)
    X/= (np.max(X)-np.min(X))

    vclust = ValidClust(
    k=list(range(2, 8)),
    methods=['hierarchical', 'kmeans'],
    indices=['silhouette', 'calinski', 'davies', 'dunn', 'cop'])
    cvi_vals = vclust.fit_predict(X)


    keys = cvi_vals[2]['kmeans'].keys()
    for method in ['hierarchical','kmeans']:
        results['data'][method]={}   
        for key in keys:
            if key=='davies':  
                idx=np.argmin([cvi_vals[x][method][key] for x in range(2,7)])
            else:
                idx=np.argmax([cvi_vals[x][method][key] for x in range(2,7)])
            #print(cvi_vals[2+idx][method][key])
            results['data'][method][key]=cvi_vals[2+idx][method][key]
    
    wdist = np.zeros((nday,nday))
    for i in range(nday):
        for j in range(i+1):
            wdist[i,j] = wd(X[i,].reshape(1,1024), X[j,].reshape(1,1024), order=2) #wasserstein(Di, Dj) # where the format of PD is numpy matrix
            wdist[j,i] = wdist[i,j]
    
    wdist=(wdist-np.min(wdist))/(np.max(wdist)-np.min(wdist))

    eps={'min':0.01, 'max':10, 'inc': 0.05}
    results['data']['DBSCAN']={}
    params['data']['DBSCAN']={}
    clusters['data']['DBSCAN']={}

    scores=['silhouette', 'calinski', 'davies', 'dunn', 'cop']
    bMax= True
    distType=0
    for score in scores:
        print('======' + score + '======' )
        if(score=='silhouette'):
            f= lambda data, labels: silhouette_score(data, labels)
        elif(score=='calinski'):
            f= lambda data, labels: calinski_harabasz_score(data, labels)
        elif(score=='davies'):
            f= lambda data, labels: davies_bouldin_score(data, labels)
            bMax=False
        elif(score == 'dunn'):
            f= lambda dist, labels: dunn(dist, labels)
            distType =1
        elif(score == 'cop'):
            f= lambda data, dist, labels: cop(data, dist, labels)
            distType =2
        results['data']['DBSCAN'][score] = {}
        params['data']['DBSCAN'][score] = {}
        clusters['data']['DBSCAN'][score] = {}
        #[score_, eps_, min_samples_, clusters_]= optimalDBSCAN(data=X, score_=f, eps=eps, metric='euclidean', bMax=bMax, distType=distType)
        [score_, eps_, min_samples_, clusters_]= optimalDBSCAN(data=wdist, score_=f, eps=eps, metric='precomputed', bMax=bMax, distType=distType)
        
        results['data']['DBSCAN'][score]= score_
        params['data']['DBSCAN'][score]['eps'] = eps_
        params['data']['DBSCAN'][score]['min_samples'] = min_samples_
        clusters['data']['DBSCAN'][score]= clusters_
    
    results['data']['Hierarchical']={}
    params['data']['Hierarchical']={}
    clusters['data']['Hierarchical'] = {}

    scores=['silhouette', 'calinski', 'davies', 'dunn','cop']
    bMax= True
    distType=0
    for score in scores:
        print('======' + score + '======' )
        if(score=='silhouette'):
            f= lambda data, labels: silhouette_score(data, labels)
        elif(score=='calinski'):
            f= lambda data, labels: calinski_harabasz_score(data, labels)
        elif(score=='davies'):
            f= lambda data, labels: davies_bouldin_score(data, labels)
            bMax=False
        elif(score == 'dunn'):
            f= lambda dist, labels: dunn(dist, labels)
            distType =1
        elif(score == 'cop'):
            f= lambda data, dist, labels: cop(data, dist, labels)
            distType =2
        results['data']['Hierarchical'][score] = {}
        params['data']['Hierarchical'][score] = {}
        clusters['data']['Hierarchical'][score] = {}
        [score_, n, linkage, clusters_] = optimalHClust(data=wdist, score_=f, metric='precomputed', bMax=bMax, distType=distType)
        results['data']['Hierarchical'][score]=score_
        params['data']['Hierarchical'][score]['n']=n
        params['data']['Hierarchical'][score]['linkage']=linkage
        clusters['data']['Hierarchical'][score]=clusters_

    wdist = np.zeros((nday,nday))
    for i in range(nday):
        xs= np.concatenate((pd0[i][0][:], pd1[i][0][:]), axis=0)
        ys= np.concatenate((pd0[i][1][:], pd1[i][1][:]), axis=0)
        Di = np.transpose([xs,ys])
    
    for j in range(i+1):
        #print(i,j)
        xs= np.concatenate((pd0[j][0][:], pd1[j][0][:]), axis=0)
        ys= np.concatenate((pd0[j][1][:], pd1[j][1][:]), axis=0)
        Dj = np.transpose([xs,ys])
    
        wdist[i,j] = wd(Di, Dj, order=2) #wasserstein(Di, Dj) # where the format of PD is numpy matrix
        wdist[j,i] = wdist[i,j]
    
    wdist=(wdist-np.min(wdist))/(np.max(wdist)-np.min(wdist))
    eps={'min':0.01, 'max':10, 'inc': 0.005}

    results['pds']['DBSCAN'] = {}
    params['pds']['DBSCAN'] = {}
    clusters['pds']['DBSCAN'] = {}

    scores=['silhouette', 'calinski', 'davies', 'dunn','cop']
    bMax= True
    distType=0
    for score in scores:
        print('======' + score + '======' )
        if(score=='silhouette'):
            f= lambda data, labels: silhouette_score(data, labels)
        elif(score=='calinski'):
            f= lambda data, labels: calinski_harabasz_score(data, labels)
        elif(score=='davies'):
            f= lambda data, labels: davies_bouldin_score(data, labels)
            bMax=False
        elif(score == 'dunn'):
            f= lambda dist, labels: dunn(dist, labels)
            distType =1
        elif(score == 'cop'):
            f= lambda data, dist, labels: cop(data, dist, labels)
            distType =2
        results['pds']['DBSCAN'][score] = {}
        params['pds']['DBSCAN'][score] = {}
        clusters['pds']['DBSCAN'][score] = {}
        [score_, eps_, min_samples_, clusters_]= optimalDBSCAN(data=wdist, score_=f, eps=eps, metric='precomputed', bMax=bMax, distType=distType)
        results['pds']['DBSCAN'][score]= score_
        params['pds']['DBSCAN'][score]['eps'] = eps_
        params['pds']['DBSCAN'][score]['min_samples'] = min_samples_
        clusters['pds']['DBSCAN'][score] = clusters_
    
    score='calinski'
    clustering = DBSCAN(eps=params['pds']['DBSCAN'][score]['eps'], min_samples=params['pds']['DBSCAN'][score]['min_samples'], metric='precomputed')
    clustering.fit(wdist)

    results['pds']['Hierarchical'] = {}
    params['pds']['Hierarchical'] = {}
    clusters['pds']['Hierarchical'] = {}

    scores=['silhouette', 'calinski', 'davies', 'dunn','cop'] #'cop'
    bMax= True
    distType=0
    for score in scores:
        print('======' + score + '======' )
        if(score=='silhouette'):
            f= lambda data, labels: silhouette_score(data, labels)
        elif(score=='calinski'):
            f= lambda data, labels: calinski_harabasz_score(data, labels)
        elif(score=='davies'):
            f= lambda data, labels: davies_bouldin_score(data, labels)
            bMax=False
        elif(score == 'dunn'):
            f= lambda dist, labels: dunn(dist, labels)
            distType =1
        elif(score == 'cop'):
            f= lambda data, dist, labels: cop(data, dist, labels)
            distType =2
        results['pds']['Hierarchical'][score] = {}
        params['pds']['Hierarchical'][score] = {}
        clusters['pds']['Hierarchical'][score] = {}
        [score_, n, linkage, clusters_] = optimalHClust(data=wdist, score_=f, metric='precomputed', bMax=bMax, distType=distType)
        results['pds']['Hierarchical'][score]=score_
        params['pds']['Hierarchical'][score]['n']=n
        params['pds']['Hierarchical'][score]['linkage']=linkage
        clusters['pds']['Hierarchical'][score]=clusters_
    
    import json
    with open('results.json', 'w') as fp:
        json.dump(results, fp)

    print("\\hline")



    for result,val_result in results.items():
        data_type = result
        print(data_type)
        print("Data type")
        print("& Algorithm")
    
    for method, val_method in val_result.items():
        print("&" + method)
        for score in val_method.keys():
            print("&" + score)
        print("\\\\") 
        print("&")  
        for score,value in val_method.items():
            print("&" + str(round(value,3)))
        print("\\\\")   
    print("\\hline")    

    L2norm = {}
    df = pd.read_excel(fname2 + '//' + 'OR_ABI-L2-FDCC-M6_G17_s20192192006196_e20192192008569_c20192192009118.Fire_Labels.xlsx')
    targetLabel = np.array(df['fire'].tolist()[0:300])

    for data in clusters.keys():
        for method in clusters[data].keys():
            for key,clustering in clusters[data][method].items():
                totalNum = np.unique(clustering.labels_)

                dictList = []
                for permute in list(itertools.permutations(totalNum)):
                    tempDict = {}
                    for i in range(len(totalNum)):
                        tempDict[i] = permute[i]
                    dictList.append(tempDict)
                
                possibleLabels = []
                for replaceDict in dictList:
                    tempLabel = np.array([replaceDict.get(item,item)  for item in clustering.labels_])
                    #possibleLabels.append(tempLabel.tolist())
                    possibleLabels.append((tempLabel * (1/(len(totalNum)-1))).tolist())

                tempL2Norms = []
                for l in possibleLabels:
                    tempL2Norms.append(LA.norm(l - targetLabel, ord=2))

                L2norm[data + ":" + method + ":" + key] = min(tempL2Norms)

    oldModelName = list(L2norm.keys())
    newModelName = []
    for name in oldModelName:
        temp = name.split(':')[2:]
        newModelName.append(":".join(temp))

    className = ['data:DBSCAN', 'data:Hierarchical', 'pis:DBSCAN', 'pis:Hierarchical', 'pds:DBSCAN', 'pds:Hierarchical']
    className = np.repeat(className, 5)

    data = {'modelName' : oldModelName,
            'largeClass' : className,
            'modelTargetL2Norm' : list(L2norm.values())}

    df = pd.DataFrame(data)

    sns.set_color_codes("pastel")
    qualitative_colors_deep = sns.color_palette('deep')
    qualitative_colors_light = sns.color_palette('pastel')
    palette = {
        'data:DBSCAN' : qualitative_colors_deep[0],
        'data:Hierarchical' : qualitative_colors_light[0],
        'pis:DBSCAN' : qualitative_colors_deep[1],
        'pis:Hierarchical' : qualitative_colors_light[1],
        'pds:DBSCAN' : qualitative_colors_deep[2],
        'pds:Hierarchical' : qualitative_colors_light[2],
    }

    f, ax = plt.subplots(figsize=(21, 9))
    ax = sns.barplot(x=oldModelName, y='modelTargetL2Norm', data=df, hue='largeClass', dodge=False, palette=palette)
    #ax = sns.barplot(x=oldModelName, y='modelTargetL2Norm', data=df, hue='largeClass', dodge=False)
    plt.title("Fire detection on Channel {}".format(channelNum))
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
    ax.legend(ncol=3, loc="upper right", frameon=True)
    ax.set(xlabel=None)
    ax.set(ylabel=None)
    ax.set_xticklabels(newModelName)
    #for p in ax.patches:
    #    ax.annotate("%.2f" % p.get_width(), xy=(p.get_width(), p.get_y()+p.get_height()/2),
    #            xytext=(5, 0), textcoords='offset points', ha="left", va="center")

    sns.despine(left=True, bottom=True)
    ax.get_figure().autofmt_xdate()
    plt.savefig("Fire detection on Channel {}.png".format(channelNum))
    files.download("Fire detection on Channel {}.png".format(channelNum)) 

0
975
{'Tile_09_38.zarr': 'Tile_9_38.zarr', 'Tile_09_35.zarr': 'Tile_9_35.zarr', 'Tile_09_05.zarr': 'Tile_9_5.zarr', 'Tile_09_04.zarr': 'Tile_9_4.zarr', 'Tile_09_36.zarr': 'Tile_9_36.zarr', 'Tile_09_06.zarr': 'Tile_9_6.zarr', 'Tile_09_09.zarr': 'Tile_9_9.zarr', 'Tile_09_07.zarr': 'Tile_9_7.zarr', 'Tile_09_37.zarr': 'Tile_9_37.zarr', 'Tile_09_08.zarr': 'Tile_9_8.zarr', 'Tile_09_34.zarr': 'Tile_9_34.zarr', 'Tile_09_33.zarr': 'Tile_9_33.zarr', 'Tile_09_28.zarr': 'Tile_9_28.zarr', 'Tile_09_30.zarr': 'Tile_9_30.zarr', 'Tile_09_29.zarr': 'Tile_9_29.zarr', 'Tile_09_03.zarr': 'Tile_9_3.zarr', 'Tile_09_32.zarr': 'Tile_9_32.zarr', 'Tile_09_31.zarr': 'Tile_9_31.zarr', 'Tile_09_26.zarr': 'Tile_9_26.zarr', 'Tile_09_27.zarr': 'Tile_9_27.zarr', 'Tile_09_23.zarr': 'Tile_9_23.zarr', 'Tile_09_25.zarr': 'Tile_9_25.zarr', 'Tile_09_21.zarr': 'Tile_9_21.zarr', 'Tile_09_17.zarr': 'Tile_9_17.zarr', 'Tile_09_22.zarr': 'Tile_9_22.zarr', 'Tile_09_18.zarr': 'Tile_9_18.zarr', 'Tile_09_19.zarr': 'Tile_9_19.zarr', '

ValueError: ignored

In [14]:
clusters['pds']['DBSCAN']['dunn'].labels_

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1])